In [1]:
import os

import pytest
import pickle
import copy

from lasso import Parameters, transit
from lasso import emme
from network_wrangler import TransitNetwork
from lasso import StandardTransit

In [2]:
%load_ext autoreload
%autoreload 2

# Remote I/O

In [3]:
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso/"

input_dir = os.path.join(lasso_dir, 'examples', 'mtc')

In [4]:
output_dir = os.path.join(lasso_dir, 'tests', 'scratch')

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-09-29 12:05:46, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso/
2021-09-29 12:05:46, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso/


## union network files

In [11]:
# roadway
union_city_links_df = pickle.load(open(os.path.join(input_dir, "union_city_links.pickle"), 'rb'))
union_city_nodes_df = pickle.load(open(os.path.join(input_dir, "union_city_nodes.pickle"), 'rb'))

# transit
transit_net = TransitNetwork.read(feed_path = input_dir)

# transit netowrk for union city
agency_df = transit_net.feed.agency.loc[transit_net.feed.agency['agency_id'] == 'UCT']
routes_df = transit_net.feed.routes.loc[transit_net.feed.routes['agency_id'] == 'UCT']
trips_df = transit_net.feed.trips[transit_net.feed.trips['route_id'].isin(routes_df['route_id'])]
freqs_df = transit_net.feed.frequencies[transit_net.feed.frequencies['trip_id'].isin(trips_df['trip_id'])]
shapes_df = transit_net.feed.shapes[transit_net.feed.shapes['shape_id'].isin(trips_df['shape_id'])]
stop_times_df = transit_net.feed.stop_times[transit_net.feed.stop_times['trip_id'].isin(trips_df['trip_id'])]
stops_df = transit_net.feed.stops[transit_net.feed.stops['stop_id'].isin(stop_times_df['stop_id'])]

union_city_transit_net =  copy.deepcopy(transit_net)
union_city_transit_net.feed.agency = agency_df
union_city_transit_net.feed.routes = routes_df
union_city_transit_net.feed.trips  = trips_df
union_city_transit_net.feed.frequencies = freqs_df
union_city_transit_net.feed.shapes = shapes_df
union_city_transit_net.feed.stop_times = stop_times_df
union_city_transit_net.feed.stops = stops_df

model_transit_net = StandardTransit.fromTransitNetwork(union_city_transit_net, parameters = parameters)

2021-09-29 12:10:49, INFO: Read in transit feed from: Z:/Data/Users/Sijia/MTC/github/Lasso/examples\mtc
2021-09-29 12:10:49, INFO: Removing calendar.txt from transit network config because file not found
2021-09-29 12:10:49, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-09-29 12:10:52, INFO: Removing transfers.txt from transit network config because file not found
2021-09-29 12:10:52, INFO: Removing feed_info.txt from transit network config because file not found
2021-09-29 12:10:52, WARNING: The following stop_ids from are referenced but missing from stops.txt: ['place_HRSH', 'place_4SSP', 'ctsmat', 'place_PSOT', 'place_HBBA', 'place_FRBA', 'place_CCCO', 'place_CHCO', 'place_CSSH', 'place_12BD', 'ctsa', 'place_TAPI', 'place_CBBA', 'place_BYBA', 'ct22', 'place_CVBA', 'ctmh', 'place_MBLS', 'ctgi', 'ctmp', 'place_14BD', 'place_DBBA', 'place_HIWY', 'ctrwc', 'ctsj', 'place_11BD', 'place_BNTE', 'place_WOBA', 'place_MOME', 'ctsu', 'ctsb', 'ctssf', 

# Write Emme Networks

In [12]:
emme.create_emme_network(
    links_df=union_city_links_df,
    nodes_df=union_city_nodes_df,
    transit_network=model_transit_net,
    name="test lasso",
    path=output_dir,
    write_taz_drive_network = False,
    write_maz_drive_network = False,
    write_maz_active_modes_network = False,
    write_tap_transit_network = True,
    parameters=parameters
)

2021-09-29 12:11:09, INFO: Converting GTFS Standard Properties to MTC's Emme Standard
2021-09-29 12:11:09, INFO: Reading faresystem from Z:/Data/Users/Sijia/MTC/github/Lasso/mtc_data\lookups\faresystem_crosswalk.txt
z:\data\users\sijia\mtc\github\lasso\tests\scratch\tap_transit_network\database\emmebank
